#Project 1 NLA: direct methods in optimization with constraints

By: Iris Vukovic

##Solving the KKT System

####C1: Write down a routine function that implements the step-size substep.

In [2]:
import numpy as np
import random
import time

In [3]:
def Newton_step(lamb0,dlamb,s0,ds):
  '''
  Compute alpha for stepsize correction substep
  lamb0: lagrange multipliers aka current parameter values
  dlamb: gradient/update direction of parameters
  s0: slack variables
  ds: gradient/update direction of slack variables
  '''
  alp = 1 #initial stepsize

  #find where gradient of lamb0 is negative
  idx_lamb0 = np.array(np.where(dlamb<0))

  if idx_lamb0.size > 0:
    alp = min(alp,np.min(-lamb0[idx_lamb0]/dlamb[idx_lamb0])) #adjust stepsize to keep lamb0 positive

  #find where gradient of s0 is negative
  idx_s0 = np.array(np.where(ds<0))

  if idx_s0.size > 0:
    alp = min(alp,np.min(-s0[idx_s0]/ds[idx_s0])) #adjust stepsize to keep s0 positive

  #alp *= 0.5

  return alp #smallest valid stepsize for lamb0 and s0

###2.1 Inequality constraints case (i.e. with $A = 0$)

####C2: Write down a program that, for a given n, implements the full algorithm for the test problem. Use the numpy.linalg.solve function to solve the KKT linear systems of the predictor and corrector substeps directly.

1. Predictor substep: We compute the standard Newton step $δz$ by solving the KKT system.

2. Step-size correction substep: Compute $α$ so that the step-size $αδ_z$ guarantees feasibility.

3. Compute $μ = \frac{s^T_0λ_0}{m}$, $\tilde{μ} =\frac{(s_0+αδ_s)^T (λ_0+αδ_λ)}{m}$ and $σ =(\tilde{μ}/μ)^3$.

4. Corrector substep: Solve the linear system with the same KKT matrix $M_{KKT}$ and right hand vector $(−\hat{r}_L, −\hat{r}_A, −\hat{r}_C, −\hat{r}_s)$ = $(−r_L, −r_A, −r_C, −r_s − D_sD_λe + σμe)$, where $D_s = diag(δ_s) and Dλ = diag(δ_λ)$.

5. Step-size correction substep.

6. Update substep: Define $z_1 = z_0 + 0.95 α δ_z$ and update $M_{KKT}$ and right-hand parts.

Now to organize into a program:

In [22]:
#define test problem
#set n to an integer value
def initialize_problem(n):

  m = 2*n #number of constraints
  G = np.identity(n)
  C = np.concatenate((np.identity(n), -np.identity(n)), axis = 1)
  g = np.random.normal(0, 1, n)
  d = np.full(m, -10)
  x0 = np.full(n, 0.0)
  s0 = lamb0 = np.full(m, 1.0)
  #gamma0 = np.zeros(n)

  S = np.diag([1] * m)
  Lambda = np.diag([1] * m)

  z0 = np.concatenate([x0, lamb0, s0])

  #solution should be x = -g

  MKKT = np.block([
      [G, -C, np.zeros((n, m))],
      [-C.T, np.zeros((m,m)), np.eye(m)],
      [np.zeros((m, n)), S, Lambda]
  ])

  return m, G, C, d, g, x0, s0, lamb0, z0, MKKT


def F_zk(G, x, g, C, lamb, d, s):
    """
    Calculates the residuals of the KKT conditions.

    Parameters:
        G, x, g, gamma, C, lamb, d, s: Variables and parameters of the problem

    Returns:
        np.ndarray: Concatenated residuals
    """
    return np.concatenate([
        G @ x + g - C @ lamb,  # Gx + g - Cλ = 0
        s + d - C.T @ x,       # s + d - C^T x = 0
        s * lamb               # siλi = 0 for all i
    ])

def algorithm(n, max_iterations=100, epsilon=1e-16):

  m, G, C, d, g, x0, s0, lamb0, z0, MKKT = initialize_problem(n)

  iteration = 0

  while iteration < max_iterations:

    #1)predictor substep
    res = F_zk(G, x0, g, C, lamb0, d, s0)

    rL = -res[:n]
    rC = -res[n:n+m]
    rs = -res[n+m:]

    r = -res  # To find the correction step

    delta_z = np.linalg.solve(MKKT, r)

    delta_lambda = delta_z[n:n+m]  # Next m elements for δλ
    delta_s = delta_z[n+m:n+m+m]  # Next m elements for δs

    #2)step-size correction substep
    alp = Newton_step(lamb0,delta_lambda,s0,delta_s)

    #3)compute values
    mu = np.dot(s0.T, lamb0) / m #avg product of slack variables and lagrange multipliers
    tilde_mu = np.dot((s0 + alp * delta_s).T, lamb0 + alp * delta_lambda) / m #average product of the updated slack variables and multipliers
    sigma = (tilde_mu / mu) ** 3 #cubic measure of the ratio of the updated average activity of the constraints (tilde mu) to the original average activity (mu)

    Ds = np.diag(delta_s)
    Dlambda = np.diag(delta_lambda)

    #4)corrector subsetp
    rhat = np.concatenate([
            rL,
            rC,
            rs - Ds @ Dlambda @ np.ones(m) + sigma * mu * np.ones(m)
        ])

    delta_z_update = np.linalg.solve(MKKT, rhat)


    delta_x_update = delta_z_update[:n]
    delta_gamma_update = delta_z_update[n:n+n]
    delta_lambda_update = delta_z_update[n:n+m]  # Next m elements for δλ
    delta_s_update = delta_z_update[n+m:n+m+m]  # Next m elements for δs


    #5)step size corection substep
    alp = Newton_step(lamb0,delta_lambda_update,s0,delta_s_update)

    #6)update substep
    x0 += 0.95 * alp * delta_x_update
    #gamma0 += 0.95 * alp * delta_gamma_update
    lamb0 += 0.95 * alp * delta_lambda_update
    s0 += 0.95 * alp * delta_s_update

    # Ensure feasibility (non-negative values for lambda and s)
    lamb0 = np.maximum(lamb0, 0)
    s0 = np.maximum(s0, 0)

    #update MKKT
    MKKT = np.block([
      [G, -C, np.zeros((n, m))],
      [-C.T, np.zeros((m,m)), np.eye(m)],
      [np.zeros((m, n)), np.diag(s0), np.diag(lamb0)]])

    #Update mu
    mu = np.dot(s0.T, lamb0) / m

    #print(f"Iteration {iteration}: x={x0}, gamma={gamma0}, lambda={lamb0}, s={s0}")
    #print(f"Residuals: rL={rL}, rC={rC}, rs={rs}, mu={np.abs(mu)}]")


    if np.any(s0 < 0) or np.any(lamb0 < 0):
      print("Warning: s0 or lamb0 went negative after update!")
      break

    if np.linalg.norm(rL) < epsilon and np.linalg.norm(rC) < epsilon and np.abs(mu) < epsilon:
      break

    # Check if x converges to -g
    if np.linalg.norm(x0 + g) < epsilon:
        print("Converged to solution.")
        break

    iteration += 1

  #return x and -g
  return {
        'x': x0,
        '-g':-g,
        'iterations': iteration
    }

algorithm(10)

Converged to solution.


{'x': array([-1.768545  , -0.14272472, -0.16064257, -0.18715857, -0.42836884,
        -0.49434075, -0.70410588, -0.37868439, -1.20700231,  2.4222147 ]),
 '-g': array([-1.768545  , -0.14272472, -0.16064257, -0.18715857, -0.42836884,
        -0.49434075, -0.70410588, -0.37868439, -1.20700231,  2.4222147 ]),
 'iterations': 15}

####C3: Write a modification of the previous program C2 to report the computation time of the solution of the test problem for different dimensions n.

In [23]:
#algorithm with time
def algorithm_time(n, max_iterations=100, epsilon=1e-16):
  start_time = time.time()

  m, G, C, d, g, x0, s0, lamb0, z0, MKKT = initialize_problem(n)

  iteration = 0

  while iteration < max_iterations:

    #1)predictor substep
    res = F_zk(G, x0, g, C, lamb0, d, s0)


    rL = -res[:n]
    rC = -res[n:n+m]
    rs = -res[n+m:]

    r = -res  # To find the correction step

    delta_z = np.linalg.solve(MKKT, r)

    delta_lambda = delta_z[n:n+m]  # Next m elements for δλ
    delta_s = delta_z[n+m:n+m+m]  # Next m elements for δs

    #2)step-size correction substep
    alp = Newton_step(lamb0,delta_lambda,s0,delta_s)

    #3)compute values
    mu = np.dot(s0.T, lamb0) / m #avg product of slack variables and lagrange multipliers
    tilde_mu = np.dot((s0 + alp * delta_s).T, lamb0 + alp * delta_lambda) / m #average product of the updated slack variables and multipliers
    sigma = (tilde_mu / mu) ** 3 #cubic measure of the ratio of the updated average activity of the constraints (tilde mu) to the original average activity (mu)

    Ds = np.diag(delta_s)
    Dlambda = np.diag(delta_lambda)

    #4)corrector subsetp
    rhat = np.concatenate([
            rL,
            rC,
            rs - Ds @ Dlambda @ np.ones(m) + sigma * mu * np.ones(m)
        ])

    delta_z_update = np.linalg.solve(MKKT, rhat)


    delta_x_update = delta_z_update[:n]
    delta_lambda_update = delta_z_update[n:n+m]  # Next m elements for δλ
    delta_s_update = delta_z_update[n+m:n+m+m]  # Next m elements for δs


    #5)step size corection substep
    alp = Newton_step(lamb0,delta_lambda_update,s0,delta_s_update)

    #6)update substep
    x0 += 0.95 * alp * delta_x_update
    lamb0 += 0.95 * alp * delta_lambda_update
    s0 += 0.95 * alp * delta_s_update

    #Ensure feasibility
    lamb0 = np.maximum(lamb0, 0) #ensures all elements of lambda are non-negative; replaces with 0 if negative
    s0 = np.maximum(s0, 0)

    #update MKKT
    MKKT = np.block([
      [G, -C, np.zeros((n, m))],
      [-C.T, np.zeros((m,m)), np.eye(m)],
      [np.zeros((m, n)), np.diag(s0), np.diag(lamb0)]])

    #Update mu
    mu = np.dot(s0.T, lamb0) / m

    #print(f"Iteration {iteration}: x={x0}, gamma={gamma0}, lambda={lamb0}, s={s0}")
    #print(f"Residuals: rL={rL}, rC={rC}, rs={rs}, mu={np.abs(mu)}]")


    if np.any(s0 < 0) or np.any(lamb0 < 0):
      print("Warning: s0 or lamb0 went negative after update!")
      break

    if np.linalg.norm(rL) < epsilon and np.linalg.norm(rC) < epsilon and np.abs(mu) < epsilon:
      break

    # Check if x converges to -g
    if np.linalg.norm(x0 + g) < epsilon:
        print("Converged to solution.")
        break

    iteration += 1

  print(f"x0 = {x0}")
  print(f"-g = {-g}")
  end_time = time.time()

  #return x and -g
  return {
        'x': x0,
        '-g' : -g,
        'iterations': iteration,
        'time': end_time - start_time}

algorithm_time(8)

Converged to solution.
x0 = [ 1.65348712  0.05756613 -0.04447366 -0.3451255  -1.2851554   1.02204454
  0.09379156 -0.69727652]
-g = [ 1.65348712  0.05756613 -0.04447366 -0.3451255  -1.2851554   1.02204454
  0.09379156 -0.69727652]


{'x': array([ 1.65348712,  0.05756613, -0.04447366, -0.3451255 , -1.2851554 ,
         1.02204454,  0.09379156, -0.69727652]),
 '-g': array([ 1.65348712,  0.05756613, -0.04447366, -0.3451255 , -1.2851554 ,
         1.02204454,  0.09379156, -0.69727652]),
 'iterations': 16,
 'time': 0.051985740661621094}

To utilize these strategies, we must assume that for (1) $\Lambda$ is invertible and for (2) that $S$ is invertible. To solve the system using $LDL^T$ factorization or Cholesky factorizattion, $G$ must be positive definite aka having all positive eigenvalues otherwise the factorization will fail.

####C4: Write down two programs (modifications of C2) that solve the optimization problem for the test problem using the previous strategies. Report the computational time for different values of n and compare with the results in C3.

In [25]:
#Strategy 1
#define test problem
#set n to an integer value
def initialize_problem(n):

  m = 2*n #number of constraints
  G = np.identity(n)
  C = np.concatenate((np.identity(n), -np.identity(n)), axis = 1)
  g = np.random.normal(0, 1, n)
  d = np.full(m, -10)
  x0 = np.full(n, 0.0)
  s0 = lamb0 = np.full(m, 1.0)

  S = np.diag([1] * m)
  Lambda = np.diag([1] * m)

  z0 = np.concatenate([x0,lamb0, s0])

  #solution should be x = -g

  MKKT = np.block([
      [G, -C],
      [-C.T, -np.linalg.inv(Lambda)@S]
  ])

  return m, G, C, d, g, x0, s0, lamb0, z0, MKKT, Lambda, S


def F_zk(G, x, g, C, lamb, d, s):
    """
    Calculates the residuals of the KKT conditions.

    Parameters:
        G, x, g, gamma, C, lamb, d, s: Variables and parameters of the problem

    Returns:
        np.ndarray: Concatenated residuals
    """
    return np.concatenate([
        G @ x + g - C @ lamb,  # Gx + g - Cλ = 0
        s + d - C.T @ x,       # s + d - C^T x = 0
        s * lamb               # siλi = 0 for all i
    ])


def LDLT_factorization(M, epsilon=1e-6):
    """
    Perform LDLT factorization of a symmetric matrix M.
    Decomposes M into L and D such that M = L * D * L^T.
    """
    n = M.shape[0]

    # Initialize L and D matrices
    L = np.zeros_like(M)
    D = np.zeros_like(M)

    for i in range(n):
        # Compute the diagonal element of D[i, i]
        D[i, i] = M[i, i] - np.dot(L[i, :i], L[i, :i] * D[:i, i])

        # Regularize if the diagonal is small or negative (to ensure positive definiteness)
        if D[i, i] < epsilon:
            D[i, i] = epsilon

        # Set L[i, i] to 1
        L[i, i] = 1

        # Compute the off-diagonal elements L[j, i] for j > i
        for j in range(i + 1, n):
            L[j, i] = (M[j, i] - np.dot(L[j, :i], L[i, :i] * D[:i, i])) / D[i, i]

    return L, D


def algorithm_strat1(n, max_iterations=100, epsilon=1e-16):
  start_time = time.time()

  m, G, C, d, g, x0, s0, lamb0, z0, MKKT, Lambda, S = initialize_problem(n)

  iteration = 0

  while iteration < max_iterations:

    #1)predictor substep
    res = F_zk(G, x0, g, C, lamb0, d, s0)

    rL = -res[:n]
    rC = -res[n:n+m]
    rs = -res[n+m:]

    r = -np.concatenate([rL, rC + np.linalg.inv(Lambda) @ rs])  # To find the correction

    # Debugging: Check for NaNs in residuals
    if np.any(np.isnan(res)):
        print(f"NaN found in residuals at iteration {iteration}")
        break

    #2)LDLT factorization and solving the system
    L, D = LDLT_factorization(MKKT)

    y = np.linalg.solve(L, r)  # Forward substitution
    delta_z = np.linalg.solve(D, y)  # Solve diagonal system
    delta_z = np.linalg.solve(L.T, delta_z)  # Back substitution

    delta_lambda = delta_z[n:n+m]  # Next m elements for δλ
    delta_s = np.linalg.inv(Lambda) @ (-rs - S @ delta_lambda)

    #2)step-size correction substep
    alp = Newton_step(lamb0,delta_lambda,s0,delta_s)

    #3)compute values
    mu = np.dot(s0.T, lamb0) / m #avg product of slack variables and lagrange multipliers
    tilde_mu = np.dot((s0 + alp * delta_s).T, lamb0 + alp * delta_lambda) / m #average product of the updated slack variables and multipliers
    sigma = (tilde_mu / mu) ** 3 #cubic measure of the ratio of the updated average activity of the constraints (tilde mu) to the original average activity (mu)

    Ds = np.diag(delta_s)
    Dlambda = np.diag(delta_lambda)

    #4)corrector subsetp
    rhat = np.concatenate([
            rL,
            rC - np.linalg.inv(Lambda)@(rs - Ds @ Dlambda @ np.ones(m) + sigma * mu * np.ones(m))
        ])

    #L, D, perm = ldl(MKKT)
    L, D = LDLT_factorization(MKKT)
    y = np.linalg.solve(L, rhat)  # Forward substitution
    delta_z_update = np.linalg.solve(D, y)  # Solve diagonal system
    delta_z_update = np.linalg.solve(L.T, delta_z_update)  # Back substitution

    delta_x_update = delta_z_update[:n]
    #delta_gamma_update = delta_z_update[n:n+n]
    delta_lambda_update = delta_z_update[n:n+m]  # Next m elements for δλ
    delta_s_update = np.linalg.inv(Lambda) @ ((rs - Ds @ Dlambda @ np.ones(m) + sigma * mu * np.ones(m)) - S @ delta_lambda_update)

    #5)step size corection substep
    alp = Newton_step(lamb0,delta_lambda_update,s0,delta_s_update)

    #6)update substep
    x0 += 0.95 * alp * delta_x_update
    #gamma0 += 0.95 * alp * delta_gamma_update
    lamb0 += 0.95 * alp * delta_lambda_update
    s0 += 0.95 * alp * delta_s_update

    #Ensure feasibility
    lamb0 = np.maximum(lamb0, 0) #ensures all elements of lambda are non-negative; replaces with 0 if negative
    s0 = np.maximum(s0, 0)

    #update MKKT
    MKKT = np.block([
      [G, -C],
      [-C.T, np.linalg.inv(Lambda)@S]])

    #Update mu
    mu = np.dot(s0.T, lamb0) / m

    #print(f"Iteration {iteration}: x={x0}, lambda={lamb0}, s={s0}")
    #print(f"Residuals: rL={rL}, rC={rC}, rs={rs}, mu={np.abs(mu)}]")


    if np.any(s0 < 0) or np.any(lamb0 < 0):
      print("Warning: s0 or lamb0 went negative after update!")
      break

    if np.linalg.norm(rL) < epsilon and np.linalg.norm(rC) < epsilon and np.abs(mu) < epsilon:
      print("Converged to solution.")
      break

    # Check if x converges to -g
    if np.linalg.norm(x0 + g) < epsilon:
        print("Converged to solution.")
        break

    iteration += 1

  end_time = time.time()

  #return x and -g
  return {
        'x': x0,
        '-g' : -g,
        'iterations': iteration,
        'time': end_time - start_time}

algorithm_strat1(2)

{'x': array([0.19894935, 1.16476681]),
 '-g': array([-0.31514376, -1.84403269]),
 'iterations': 100,
 'time': 0.5891709327697754}

I'm not getting $x=-g$ in Strategy 1, but I'm relatively close.

In [28]:
#Strategy 2
def initialize_problem(n):

  m = 2*n #number of constraints
  G = np.identity(n)
  C = np.concatenate((np.identity(n), -np.identity(n)), axis = 1)
  g = np.random.normal(0, 1, n)
  d = np.full(m, -10)
  x0 = np.full(n, 0.0)
  s0 = lamb0 = np.full(m, 1.0)

  S = np.diag([1] * m)
  Lambda = np.diag([1] * m)

  z0 = np.concatenate([x0, lamb0, s0])

  #solution should be x = -g

  MKKT = np.block([
      [G + C @ np.linalg.inv(S) @ Lambda @ C.T]
  ])

  return m, G, C, d, g, x0, s0, lamb0, z0, MKKT, S, Lambda


def F_zk(G, x, g, C, lamb, d, s):
    """
    Calculates the residuals of the KKT conditions
    """
    return np.concatenate([
        G @ x + g - C @ lamb,  # Gx + g - Cλ = 0
        s + d - C.T @ x,       # s + d - C^T x = 0
        s * lamb               # siλi = 0 for all i
    ])

def cholesky_decomposition(A):
    """
    Perform Cholesky decomposition of a positive-definite matrix A.
    Decomposes A into L, where A = L * L.T.
    """
    n = A.shape[0]
    L = np.zeros_like(A)

    for i in range(n):
        for j in range(i + 1):
            sum_ = np.dot(L[i, :j], L[j, :j])  # Dot product of the current row/column up to j
            if i == j:
                # Diagonal elements
                L[i, j] = np.sqrt(A[i, i] - sum_)
            else:
                # Off-diagonal elements
                L[i, j] = (A[i, j] - sum_) / L[j, j]

    return L


def cholesky_solve(L, b):
    """
    Solve the system of equations A * x = b using Cholesky decomposition.
    For A = L * L.T, solve L * L.T * x = b.
    """
    # Solve L * y = b
    y = np.linalg.solve(L, b)

    # Solve L.T * x = y
    x = np.linalg.solve(L.T, y)

    return x


def algorithm_strat2(n, max_iterations=100, epsilon=1e-16):
  start_time = time.time()

  m, G, C, d, g, x0, s0, lamb0, z0, MKKT, S, Lambda = initialize_problem(n)

  iteration = 0

  while iteration < max_iterations:

    #1)predictor substep
    res = F_zk(G, x0, g, C, lamb0, d, s0)

    rL = -res[:n]
    rC = -res[n:n+m]
    rs = -res[n+m:]
    rhat = -C @ np.linalg.inv(S) @ (-rs + Lambda @ rC)
    rhs = rL - rhat

    L = cholesky_decomposition(MKKT)

    delta_x = cholesky_solve(L, rhs)

    delta_lambda = np.linalg.inv(S)@(-rs + Lambda @ rC) - np.linalg.inv(S) @ Lambda @ C.T @ delta_x
    delta_s = -rC + C.T @ delta_x


    #2)step-size correction substep
    alp = Newton_step(lamb0,delta_lambda,s0,delta_s)

    #3)compute values
    mu = np.dot(s0.T, lamb0) / m #avg product of slack variables and lagrange multipliers
    tilde_mu = np.dot((s0 + alp * delta_s).T, lamb0 + alp * delta_lambda) / m #average product of the updated slack variables and multipliers
    sigma = (tilde_mu / mu) ** 3 #cubic measure of the ratio of the updated average activity of the constraints (tilde mu) to the original average activity (mu)

    Ds = np.diag(delta_s)
    Dlambda = np.diag(delta_lambda)

    #4)corrector subset
    rhat_update = -C @ np.linalg.inv(S) @ (-rs - Ds @ Dlambda @ np.ones(m) + sigma * mu * np.ones(m) + Lambda @ rC)
    rhs_update = rL - rhat_update

    #L = cholesky_decomposition(MKKT)
    delta_x_update = cholesky_solve(L, rhs_update)

    delta_lambda_update = np.linalg.inv(S)@((-rs - Ds @ Dlambda @ np.ones(m) + sigma * mu * np.ones(m)) + Lambda @ rC) - np.linalg.inv(S) @ Lambda @ C.T @ delta_x_update
    delta_s_update = -rC + C.T @ delta_x_update


    #5)step size corection substep
    alp = Newton_step(lamb0,delta_lambda_update,s0,delta_s_update)

    #6)update substep
    x0 += 0.95 * alp * delta_x_update
    lamb0 += 0.95 * alp * delta_lambda_update
    s0 += 0.95 * alp * delta_s_update

    # Ensure feasibility (non-negative values for lambda and s)
    lamb0 = np.maximum(lamb0, 0)
    s0 = np.maximum(s0, 0)

    #update MKKT
    MKKT = np.block([
      [G + C @ np.linalg.inv(S) @ Lambda @ C.T]
  ])

    #Update mu
    mu = np.dot(s0.T, lamb0) / m

    #print(f"Iteration {iteration}: x={x0}, lambda={lamb0}, s={s0}")
    #print(f"Residuals: rL={rL}, rC={rC}, rs={rs}, mu={np.abs(mu)}]")


    if np.any(s0 < 0) or np.any(lamb0 < 0):
      print("Warning: s0 or lamb0 went negative after update!")
      break

    if np.linalg.norm(rL) < epsilon and np.linalg.norm(rC) < epsilon and np.abs(mu) < epsilon:
      break

    # Check if x converges to -g
    if np.linalg.norm(x0 + g) < epsilon:
        print("Converged to solution.")
        break

    iteration += 1


  end_time = time.time()

  #return z0
  return {
        'x': x0,
        '-g' : -g,
        'iterations': iteration,
        'time': end_time - start_time}

algorithm_strat2(2)

{'x': array([-0.37888177,  7.49211098]),
 '-g': array([-0.36267111,  0.78198653]),
 'iterations': 100,
 'time': 0.24561095237731934}

In Strategy 2, my $x$ and $-g$ are not equal and not very close.

I realize that with the two strategies, the time should be faster, but since my algorithm isn't working correctly, the time is longer.

####C5: Write down a program that solves the optimization problem for the general case. Use numpy.linalg.solve function. Read the data of the optimization problems from the files (available at the Campus Virtual). Each problem consists on a collection of files: G.dad, g.dad, A.dad, b.dad, C.dad and d.dad. They contain the corresponding data in coordinate format. Take as initial condition $x_0 = (0, . . . , 0)$ and $s_0 = γ_0 = λ_0 = (1, . . . , 1)$ for all problems.

In [8]:
A = np.loadtxt('/content/A.dad')
C = np.loadtxt('/content/C.dad')
b = np.loadtxt('/content/b.dad')
d = np.loadtxt('/content/d.dad')
G = np.loadtxt('/content/G.dad')
g = np.loadtxt('/content/g.dad')

In [9]:
n = 100
m = 200
p = 50
x0 = np.full(n, 0.0)
s0 = lamb0 = np.full(m, 1.0)
gamma0 = np.full(p, 1.0)

In [20]:
#initialize matrices with values from coordinate matrices from files
def initialize_matrix(matrix, coordinates):
  for coord in coordinates:

    row = int(coord[0]) - 1  # Convert 1-based index to 0-based
    col = int(coord[1]) - 1  # Convert 1-based index to 0-based
    if len(coord) == 3:
      value = coord[2]
      matrix[row, col] = value
    else:
      value = coord[1]
      matrix[row] = value

  return matrix

def fill_symmetric_matrix(G, upper_triangular_data):
    # Assuming upper_triangular_data is a list of tuples (i, j, value)
    # where i < j, i.e., the upper triangular part of G
    for (i, j, value) in upper_triangular_data:
        i, j = int(i) - 1, int(j) - 1
        G[i, j] = value
        G[j, i] = value  # Ensure symmetry by mirroring the value in the lower triangular part
    return G


#define test problem
#set n to an integer value
def initialize_problem(n, x0, lamb0, gamma0, s0):

  m = 2*n #number of inequality constraints
  p = n//2 #number of equality constraints
  #g = np.zeros(n)

  A = np.loadtxt('/content/A.dad')
  C = np.loadtxt('/content/C.dad')
  b = np.loadtxt('/content/b.dad')
  d = np.loadtxt('/content/d.dad')
  G = np.loadtxt('/content/G.dad')
  g = np.loadtxt('/content/g.dad')

  new_A = np.zeros((n, p))
  new_C = np.zeros((n, m))
  new_b = np.zeros(p,)
  new_d = np.zeros(m,)
  new_G = np.zeros((n,n))
  new_g = np.zeros(n,)

  A = initialize_matrix(new_A, A)
  C = initialize_matrix(new_C, C)
  b = initialize_matrix(new_b, b)
  d = initialize_matrix(new_d, d)
  g = initialize_matrix(new_g, g)
  G = fill_symmetric_matrix(new_G, G) #to make it upper triangular

  '''print('A: ' , A)
  print('C: ' , C)
  print('b: ' , b)
  print('d: ' , d)
  print('g: ' , g)
  print('G: ' , G)'''
  #print(A.shape, C.shape, b.shape, d.shape, G.shape)

  '''x0 = np.full(n, 0.0)
  s0 = lamb0 = np.full(m, 1.0)
  gamma0 = np.full(p, 1.0)'''

  S = np.diag(s0)
  Lambda = np.diag(lamb0)

  '''print(f"G.shape: {G.shape}")
  print(f"-A.shape: {A.shape}")
  print(f"-C.shape: {C.shape}")
  print(f"np.zeros((n, m)).shape: {np.zeros((n, m)).shape}")
  print(f"-A.T.shape: {A.T.shape}")
  print(f"np.zeros((n, n)).shape: {np.zeros((n, n)).shape}")
  print(f"np.zeros((n, m)).shape: {np.zeros((n, m)).shape}")
  print(f"np.eye(m).shape: {np.eye(m).shape}")
  print(f"S.shape: {S.shape}")
  print(f"Lambda.shape: {Lambda.shape}")'''

  z0 = np.concatenate([x0, lamb0, gamma0, s0])

  #The optpr1 (n = 100, p = 50, and m = 200) has as solution a vector x such that f(x) = 1.15907181 × 10^4
  #The optpr2 (n = 1000, p = 500, and m = 2000) has as solution a vector x such that f(x) = 1.08751157 × 10^6

  MKKT = np.block([
      [G, -A, -C, np.zeros((n, m))],
      [-A.T, np.zeros((p, p)), np.zeros((p, m)), np.zeros((p, m))],
      [-C.T, np.zeros((m, p)), np.zeros((m,m)), np.eye(m)],
      [np.zeros((m, n)), np.zeros((m, p)), S, Lambda]
  ])

  return m, p, G, A, b, C, d, g, z0, MKKT


def F_zk(G, x0, g, A, gamma0, C, lamb0, b, d, s0):
    """
    Calculates the residuals of the KKT conditions.

    Parameters:
        G, x, g, A, gamma, C, lamb, b, d, s: Variables and parameters of the problem

    Returns:
        np.ndarray: Concatenated residuals
    """
    return np.concatenate([
        G @ x0 + g - A @ gamma0 - C @ lamb0,  # Gx + g - Aγ - Cλ = 0
        b - A.T @ x0,                          # b - A^T x = 0
        s0 + d - C.T @ x0,                      # s + d - C^T x = 0
        s0 * lamb0                               # siλi = 0 for all i
    ])

def algorithm(n,  x0, lamb0, gamma0, s0, max_iterations=100, epsilon=1e-16):

  m, p, G, A, b, C, d, g, z0, MKKT = initialize_problem(n, x0, lamb0, gamma0, s0)
  mu = np.dot(s0.T, lamb0) / m

  iteration = 0

  while iteration < max_iterations:

    res = F_zk(G, x0, g, A, gamma0, C, lamb0, b, d, s0)

    if np.any(np.isnan(res)):
      print("Residuals contain NaNs:", res)
      break

    rL = -res[0:n]  # rL should have shape (100,)
    rA = -res[n:n+p]  # rA should have shape (50,)
    rC = -res[n+p:n+p+m]  # rC should have shape (200,)
    rs = -res[n+p+m:]

    #r = -F_zk(G, x0, g, A, gamma0, C, lamb0, b, d, s0)
    delta_z = np.linalg.solve(MKKT, -res)

    if np.any(np.isnan(delta_z)):
      print("Delta_z contains NaNs:", delta_z)
      break

    #delta_lambda = delta_z[n:n+m]  # Next m elements for δλ
    #delta_s = delta_z[n+m:n+m+m]  # Next m elements for δs

    delta_lambda = delta_z[n+p:n+p+m]  # Next m elements for δλ
    delta_s = delta_z[n+p+m:]  # Next m elements for δs


    alp = Newton_step(lamb0,delta_lambda,s0,delta_s)

    mu = np.dot(s0.T, lamb0) / m #avg product of slack variables and lagrange multipliers
    tilde_mu = np.dot((s0 + alp * delta_s).T, lamb0 + alp * delta_lambda) / m #average product of the updated slack variables and multipliers
    sigma = (tilde_mu / mu) ** 3 #cubic measure of the ratio of the updated average activity of the constraints (tilde mu) to the original average activity (mu)

    Ds = np.diag(delta_s)
    Dlambda = np.diag(delta_lambda)

    rhat = np.concatenate([
            rL,
            rA,
            rC,
            rs - Ds @ Dlambda @ np.ones(m) + sigma * mu * np.ones(m)
        ])

    if np.any(np.isnan(rhat)):
      print("rhat contains NaNs:", rhat)
      break

    delta_z_update = np.linalg.solve(MKKT, rhat)

    delta_x_update = delta_z_update[0:n]          # This updates x0
    delta_gamma_update = delta_z_update[n:n+p]   # This should update gamma0 (size p)
    delta_lambda_update = delta_z_update[n+p:n+p+m]  # This updates lamb0
    delta_s_update = delta_z_update[n+p+m:]

    alp = Newton_step(lamb0,delta_lambda_update,s0,delta_s_update)

    # Step size update
    x0 += 0.95 * alp * delta_x_update
    gamma0 += 0.95 * alp * delta_gamma_update
    lamb0 += 0.95 * alp * delta_lambda_update
    #print('lamb0: ',lamb0)
    s0 += 0.95 * alp * delta_s_update
    #print('s0: ',s0)

    S = np.diag(s0)
    #print('S: ', S)
    Lambda = np.diag(lamb0)

    #update MKKT
    MKKT = np.block([
      [G, -A, -C, np.zeros((n, m))],
      [-A.T, np.zeros((p, p)), np.zeros((p, m)), np.zeros((p, m))],
      [-C.T, np.zeros((m,p)), np.zeros((m,m)), np.eye(m)],
      [np.zeros((m, n)), np.zeros((m, p)), np.diag(s0), np.diag(lamb0)]])


    #print(MKKT)

    #print(f"Iteration {iteration}: x={x0}, gamma={gamma0}, lambda={lamb0}, s={s0}")
    #print(f"Residuals: rL={rL}, rA={rA}, rC={rC}, rs={rs}")
    #print(f"Iteration {iteration}: x={x0}")


    if np.any(s0 < 0) or np.any(lamb0 < 0):
      print("Warning: s0 or lamb0 went negative after update!")
      break

    if np.linalg.norm(rL) < epsilon and np.linalg.norm(rA) < epsilon and np.linalg.norm(rC) < epsilon and np.abs(mu) < epsilon:
      break

    iteration += 1

  #return f(x)
  f_x = 0.5 * x0.T @ G @ x0 + g.T @ x0

  return {
        'f(x)': f_x,
        'iterations': iteration
    }

algorithm(100, x0, lamb0, gamma0, s0)


{'f(x)': 9360.96482590454, 'iterations': 100}

My answer is so wrong.

####C6: Implement a routine that uses LDLT to solve the optimizations problems (in C5) and compare the results.

In [12]:
n = 100
m = 200
p = 50
x0 = np.full(n, 0.0)
s0 = lamb0 = np.full(m, 1.0)
gamma0 = np.full(p, 1.0)

In [18]:
#initialize matrices with values from coordinate matrices from files
def initialize_matrix(matrix, coordinates):
  for coord in coordinates:

    row = int(coord[0]) - 1  # Convert 1-based index to 0-based
    col = int(coord[1]) - 1  # Convert 1-based index to 0-based
    if len(coord) == 3:
      value = coord[2]
      matrix[row, col] = value
    else:
      value = coord[1]
      matrix[row] = value

  return matrix

def fill_symmetric_matrix(G, upper_triangular_data):
    # Assuming upper_triangular_data is a list of tuples (i, j, value)
    # where i < j, i.e., the upper triangular part of G
    for (i, j, value) in upper_triangular_data:
        i, j = int(i) - 1, int(j) - 1
        G[i, j] = value
        G[j, i] = value  # Ensure symmetry by mirroring the value in the lower triangular part
    return G


#define test problem
#set n to an integer value
def initialize_problem(n, x0, lamb0, gamma0, s0):

  m = 2*n #number of inequality constraints
  p = n//2 #number of equality constraints
  #g = np.zeros(n)

  A = np.loadtxt('/content/A.dad')
  C = np.loadtxt('/content/C.dad')
  b = np.loadtxt('/content/b.dad')
  d = np.loadtxt('/content/d.dad')
  G = np.loadtxt('/content/G.dad')
  g = np.loadtxt('/content/g.dad')

  new_A = np.zeros((n, p))
  new_C = np.zeros((n, m))
  new_b = np.zeros(p,)
  new_d = np.zeros(m,)
  new_G = np.zeros((n,n))
  new_g = np.zeros(n,)

  A = initialize_matrix(new_A, A)
  C = initialize_matrix(new_C, C)
  b = initialize_matrix(new_b, b)
  d = initialize_matrix(new_d, d)
  g = initialize_matrix(new_g, g)
  G = fill_symmetric_matrix(new_G, G) #to make it upper triangular

  '''print('A: ' , A)
  print('C: ' , C)
  print('b: ' , b)
  print('d: ' , d)
  print('g: ' , g)
  print('G: ' , G)'''
  #print(A.shape, C.shape, b.shape, d.shape, G.shape)

  '''x0 = np.full(n, 0.0)
  s0 = lamb0 = np.full(m, 1.0)
  gamma0 = np.full(p, 1.0)'''

  S = np.diag(s0)
  Lambda = np.diag(lamb0)

  '''print(f"G.shape: {G.shape}")
  print(f"-A.shape: {A.shape}")
  print(f"-C.shape: {C.shape}")
  print(f"np.zeros((n, m)).shape: {np.zeros((n, m)).shape}")
  print(f"-A.T.shape: {A.T.shape}")
  print(f"np.zeros((n, n)).shape: {np.zeros((n, n)).shape}")
  print(f"np.zeros((n, m)).shape: {np.zeros((n, m)).shape}")
  print(f"np.eye(m).shape: {np.eye(m).shape}")
  print(f"S.shape: {S.shape}")
  print(f"Lambda.shape: {Lambda.shape}")'''

  z0 = np.concatenate([x0, lamb0, gamma0, s0])

  #The optpr1 (n = 100, p = 50, and m = 200) has as solution a vector x such that f(x) = 1.15907181 × 10^4
  #The optpr2 (n = 1000, p = 500, and m = 2000) has as solution a vector x such that f(x) = 1.08751157 × 10^6

  MKKT = np.block([
      [G, -A, -C],
      [-A.T, np.zeros((p, p)), np.zeros((p, m))],
      [-C.T, np.zeros((m, p)), -np.linalg.inv(Lambda) @ S]
  ])

  return m, p, G, A, b, C, d, g, z0, MKKT


def F_zk(G, x0, g, A, gamma0, C, lamb0, b, d, s0):
    """
    Calculates the residuals of the KKT conditions.

    Parameters:
        G, x, g, A, gamma, C, lamb, b, d, s: Variables and parameters of the problem

    Returns:
        np.ndarray: Concatenated residuals
    """
    return np.concatenate([
        G @ x0 + g - A @ gamma0 - C @ lamb0,  # Gx + g - Aγ - Cλ = 0
        b - A.T @ x0,                          # b - A^T x = 0
        s0 + d - C.T @ x0,                      # s + d - C^T x = 0
        s0 * lamb0                               # siλi = 0 for all i
    ])

def LDLT_factorization(M, epsilon=1e-6):
    """
    Perform LDLT factorization of a symmetric matrix M.
    Decomposes M into L and D such that M = L * D * L^T.
    """
    n = M.shape[0]

    # Initialize L and D matrices
    L = np.zeros_like(M)
    D = np.zeros_like(M)

    for i in range(n):
        # Compute the diagonal element of D[i, i]
        D[i, i] = M[i, i] - np.dot(L[i, :i], L[i, :i] * D[:i, i])

        # Regularize if the diagonal is small or negative (to ensure positive definiteness)
        if D[i, i] < epsilon:
            D[i, i] = epsilon

        # Set L[i, i] to 1
        L[i, i] = 1

        # Compute the off-diagonal elements L[j, i] for j > i
        for j in range(i + 1, n):
            L[j, i] = (M[j, i] - np.dot(L[j, :i], L[i, :i] * D[:i, i])) / D[i, i]

    return L, D


def algorithm(n,  x0, lamb0, gamma0, s0, max_iterations=100, epsilon=1e-16):

  m, p, G, A, b, C, d, g, z0, MKKT = initialize_problem(n, x0, lamb0, gamma0, s0)
  mu = np.dot(s0.T, lamb0) / m

  iteration = 0

  while iteration < max_iterations:

    Lambda = np.diag(lamb0)

    res = F_zk(G, x0, g, A, gamma0, C, lamb0, b, d, s0)

    if np.any(np.isnan(res)):
      print("Residuals contain NaNs:", res)
      break

    rL = -res[0:n]  # rL should have shape (100,)
    rA = -res[n:n+p]  # rA should have shape (50,)
    rC = -res[n+p:n+p+m]  # rC should have shape (200,)
    rs = -res[n+p+m:]
    rC -= np.linalg.inv(Lambda) @ rs

    r = -np.concatenate([rL, rA, rC])

    #delta_z = np.linalg.solve(MKKT, r)

    L, D = LDLT_factorization(MKKT)

    y = np.linalg.solve(L, r)  # Forward substitution
    delta_z = np.linalg.solve(D, y)  # Solve diagonal system
    delta_z = np.linalg.solve(L.T, delta_z)  # Back substitution



    if np.any(np.isnan(delta_z)):
      print("Delta_z contains NaNs:", delta_z)
      break

    S = np.diag(s0)

    delta_lambda = delta_z[n:n+m]
    delta_s = np.linalg.inv(Lambda) @ (-rs - S @ delta_lambda)


    alp = Newton_step(lamb0,delta_lambda,s0,delta_s)

    mu = np.dot(s0.T, lamb0) / m #avg product of slack variables and lagrange multipliers
    tilde_mu = np.dot((s0 + alp * delta_s).T, lamb0 + alp * delta_lambda) / m #average product of the updated slack variables and multipliers
    sigma = (tilde_mu / mu) ** 3 #cubic measure of the ratio of the updated average activity of the constraints (tilde mu) to the original average activity (mu)

    Ds = np.diag(delta_s)
    Dlambda = np.diag(delta_lambda)

    rhat = np.concatenate([
            rL,
            rA,
            rC - np.linalg.inv(Lambda) @ (rs - Ds @ Dlambda @ np.ones(m) + sigma * mu * np.ones(m))
        ])

    if np.any(np.isnan(rhat)):
      print("rhat contains NaNs:", rhat)
      break

    y = np.linalg.solve(L, rhat)  # Forward substitution
    delta_z_update = np.linalg.solve(D, y)  # Solve diagonal system
    delta_z_update = np.linalg.solve(L.T, delta_z_update)  # Back substitution

    delta_x_update = delta_z_update[:n]          # This updates x0
    delta_gamma_update = delta_z_update[n:n+p]   # This should update gamma0 (size p)
    delta_lambda_update = delta_z_update[n+p:n+p+m]  # This updates lamb0
    delta_s_update = np.linalg.inv(Lambda) @ ((-rs - Ds @ Dlambda @ np.ones(m) + sigma * mu * np.ones(m))- S @ delta_lambda)

    alp = Newton_step(lamb0,delta_lambda_update,s0,delta_s_update)

    # Step size update
    x0 += 0.95 * alp * delta_x_update
    gamma0 += 0.95 * alp * delta_gamma_update
    lamb0 += 0.95 * alp * delta_lambda_update
    s0 += 0.95 * alp * delta_s_update

    #print(f"Iteration {iteration}: x={x0}, gamma={gamma0}, lambda={lamb0}, s={s0}")
    #print(f"Residuals: rL={rL}, rA={rA}, rC={rC}, rs={rs}")

    #update MKKT
    S = np.diag(s0)
    Lambda = np.diag(lamb0)

    MKKT = np.block([
      [G, -A, -C],
      [-A.T, np.zeros((p, p)), np.zeros((p, m))],
      [-C.T, np.zeros((m, p)), -np.linalg.inv(Lambda) @ S]
  ])


    if np.any(s0 < 0) or np.any(lamb0 < 0):
      print("Warning: s0 or lamb0 went negative after update!")
      break

    if np.linalg.norm(rL) < epsilon and np.linalg.norm(rA) < epsilon and np.linalg.norm(rC) < epsilon and np.abs(mu) < epsilon:
      break

    iteration += 1

  #return f(x)
  f_x = 0.5 * x0.T @ G @ x0 + g.T @ x0

  return {'f(x)': f_x}

algorithm(100, x0, lamb0, gamma0, s0)


{'f(x)': 9372.165069678853}

Once again, my result is very far from the desired result.

####Difficulties:
The most common issue I encountered for the duration of this project was mismatching dimensions preventing me from calculating dot products or initializing matrices. I also found that, though my algorithms run and seemingly follow all of the correct initialization and calculation steps, many of them output incorrect answers.

My algorithms aren't giving me exact results, but I think the logic behind them is correct for the most part. If I had more time, I would alter things more to get the desired results.